In [941]:
import qcodes as qc
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import qcodes.dataset.experiment_container as exc
import dash
import dash_core_components as dcc
import dash_html_components as html
from qcodes.dataset.data_set import load_by_id
from qcodes.dataset.data_export import get_data_by_id
from qcodes import load_experiment
from dash.dependencies import Input, Output, State
from plotly import tools
import plotly.io as pio
from plotly import graph_objs as go
from qcodes.instrument_drivers.rohde_schwarz import SGS100A
from time import sleep
from qcodes.dataset.measurements import Measurement
from scipy.optimize import curve_fit
import datetime
import scipy.fftpack
from scipy import signal
from scipy.signal import find_peaks
import pdfkit
from dash.exceptions import PreventUpdate

In [942]:
configuration = qc.config
print(f'Using config file from {configuration.current_config_path}')
configuration['core']['db_location'] = './2019-07-08-frays-final.db'
print(f'Database location: {configuration["core"]["db_location"]}')
qc.initialise_database()

Using config file from //anaconda3/lib/python3.7/site-packages/qcodes/config/qcodesrc.json
Database location: ./2019-07-08-frays-final.db


In [943]:
#exc.experiments()

In [944]:
def normalizeCounts(countArray, num):
    reb_counts = np.squeeze(countArray)/np.mean(np.sort(np.squeeze(countArray))[-num:])
    return reb_counts

In [1262]:
def plotdata(expt, files, normalize, plotcurrent=0, nPi = []): 
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho' or 'nmr'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    if plotcurrent == 1:        
        filesize = np.size(files) + 1
    else:
        filesize = np.size(files)  
        
    plotfun = go.Figure()    
    for i in range(filesize):
        if plotcurrent == 1 and i == filesize-1:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
        else:    
            Data2 = exc.load_by_id(files[i])
        if expt == 'spinecho':
            x2 = 2*np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo')                  
        elif expt == 'doubleecho':
            if nPi == [] or nPi[i] == 1:
                x2 = 2*np.squeeze(Data2.get_data('Time'))
            else:
                x2 = nPi[i]*np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo Double Measure')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'nmr':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='NMR')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'odmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='ODMR')
            if normalize == 1:
                y2 = normalizeCounts(Data2.get_data('Counts'),50)
                layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Normalized Counts'),title='ODMR')
        elif expt == 'pulsedodmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))   
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='Pulsed ODMR')
        elif expt == 'g2':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Norm_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time dif'), yaxis=dict(title='Normalised Counts'), title='g2 Dip')
        else:
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Rabi')
        if plotcurrent == 1 and i == filesize-1:
            plotfun.add_scatter(x = x2, y = y2, name = 'Recent Data', mode='lines+markers') 
        else:       
            plotfun.add_scatter(x = x2, y = y2, name = all_runs[files[i]-1]['label'], mode='lines+markers') 
            
    return plotfun

In [1233]:
def updatedata(plotfun, expt, sleeptime=2, normalize=1, animation=1):
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or rabi' or 'ramsey' or 'spinecho' or 'doubleecho'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    for i in range(10000):
        sleep(sleeptime)
        if animation == 1:
            with plotfun.batch_animate(easing = 'quad'):   
                Data2 = exc.load_last_experiment()
                Data2 = Data2.last_data_set()
                if expt == 'doubleecho':
                    y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                    y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                    y2 = y2ms0 - y2ms1  
                    if normalize == 1:
                        y2 = (y2 + max(y2))/(2*max(y2))
                elif expt == 'nmr':
                    y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                    y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                    y2 = y2ms0 - y2ms1  
                    if normalize == 1:
                        y2 = (y2 + max(y2))/(2*max(y2))
                elif expt == 'odmr':
                    y2 = np.squeeze(Data2.get_data('Counts'))
                    if normalize == 1:
                        y2 = normalizeCounts(Data2.get_data('Counts'),50)
                elif expt == 'g2':
                    y2 = np.squeeze(Data2.get_data('Norm_Counts'))
                else:
                    y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
                plotfun.data[-1].y = y2
        else:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
            if expt == 'doubleecho':
                y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                y2 = y2ms0 - y2ms1  
                if normalize == 1:
                    y2 = (y2 + max(y2))/(2*max(y2))
            elif expt == 'nmr':
                y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                y2 = y2ms0 - y2ms1  
                if normalize == 1:
                    y2 = (y2 + max(y2))/(2*max(y2))
            elif expt == 'odmr':
                y2 = np.squeeze(Data2.get_data('Counts'))
                if normalize == 1:
                    y2 = normalizeCounts(Data2.get_data('Counts'),50)
            elif expt == 'g2':
                y2 = np.squeeze(Data2.get_data('Counts'))
            else:
                y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.data[-1].y = y2

In [1234]:
def lorentzian(x, amp, a0, x0, g):
    denom = (x - x0)**2 + (0.5*g)**2
    num = 0.5*g
    frac = a0 - (num/denom) * (amp)/np.pi
    return frac

In [1145]:
def sinedamp(x,a,c,f,t):
    fun = a*np.cos(2*np.pi*f*x)*np.exp(-1*(x/t)) + c
    return fun

In [1146]:
def stretchedexp(x,a,c,k,t):
    fun = a*np.exp(-1*(x/t)**k) + c
    return fun

In [1237]:
def fitlorentzian(file, expt, plotfun,lb,ub):
    "Fitting parameters are Amplitude, Baseline Offset, ODMR frequency in GHz, and Width in GHz. \
    Attribute - expt - can either be 'odmr' or 'pulsedodmr'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file)
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Frequency'
    x1 = np.squeeze(Data.get_data(xaxis))
    
    if expt == 'odmr':
        yaxis = 'Counts'
        y1 = normalizeCounts(Data.get_data(yaxis),50)
    elif expt == 'pulsedodmr':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
    '''    
    index = np.argmin(y1)
    freq = x1[index]/1e9
    
    if (lb and ub) == None:
        lb = [freq-0.01, 0.95, 0.007, 0.0005]
        ub = [freq+0.01, 1.1, 0.02, 0.005]
    '''   
    print(lb,ub)
    popt, pcov = curve_fit(lorentzian, x1/1e9, y1, bounds=(lb,ub))
    fitname = str(file)+ ': Fit' + ', Resonance @ ' + str(round((popt[2]),3)) + ' GHz'
    plotfun.add_scatter(x = x1 , y = lorentzian(x1/1e9,*popt), name = fitname)
    print(plotfun)
    return plotfun,popt

In [1238]:
def fitsinedamp(file,expt,plotfun,lb,ub):   
    "Fitting parameters are Amplitude, Baseline Offset, Oscillation Frequency in GHz, and Decay Time in ns.\
    Attribute - expt - can either be 'rabi' or 'spinecho' or 'doubleecho'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file)
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Time'
    
    
    if expt == 'rabi':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
    '''    
    ampMin = 0.8*(y1.max()-y1.min())/2
    ampMax = 1.2*(y1.max()-y1.min())/2
    boMin = 0.4*(y1.max()+y1.min())
    boMax = 0.6*(y1.max()+y1.min())
    freq = 1/(x1[y1.argmin()]*2)
    freqMin = freq*0.7
    freqMax = freq*1.3
    if (lb and ub) == None:
        lb = [ampMin, boMin, freqMin, 0]
        ub = [ampMax, boMax, freqMax, 1e5]
    ''' 
    popt, pcov = curve_fit(sinedamp, x1, y1, bounds=(lb,ub))
    yval = sinedamp(x1,*popt)
    fitname = str(file)+ ': Fit' + ', \u0394 = ' + str(round((popt[0]*2*100),1)) + ' %' + ', \u03C0 = ' +  str(round((0.5/popt[2]),1)) + ' ns'
    if plotfun is not None:
        plotfun.add_scatter(x = x1 , y = yval, name = fitname,line=dict(shape='spline')) 
    return plotfun, popt, pcov

In [1239]:
def find_T2(file,expt,plotfun,threshold, lb,ub, revivals):   
    "Fitting parameters are Amplitude, Baseline Offset, Power of Stretched Exponential, and Decay Time in ns.\
    Attribute - expt - can either be 'spinecho' or 'doubleecho'.\
    threshold is required for peak detection"
    
    if np.size(file) != 0:
        Data = exc.load_by_id(file)
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    x1 = 2*np.squeeze(Data.get_data('Time'))
    if expt == 'spinecho':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
    elif expt == 'doubleecho':
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1 
        y1 = (y1 + max(y1))/(2*max(y1))
    if revivals == 1:
        peaks, _= find_peaks(y1,height=threshold,distance=6, width=3)
        x0 = np.array([0])
        y0 = np.array([y1[0]])
        xpeaks = np.concatenate([x0, x1[peaks]], axis=0)
        print(xpeaks)
        ypeaks = np.concatenate([y0, y1[peaks]], axis=0)
        print(ypeaks)
    else:
        xpeaks = x1
        ypeaks = y1
    
    popt, pcov = curve_fit(stretchedexp, xpeaks, ypeaks, bounds=(lb,ub))
    yval = stretchedexp(xpeaks,*popt)
    
    fitname = str(file)+ ': Fit, T2 = ' + str(round((popt[3]/1e3),1)) + ' \u03BCs' 
    
    #plotfun.add_scatter(x = xpeaks, y = ypeaks, mode='markers', name = 'Detected Peaks',marker=dict(color='red', size=10, opacity=0.5)) 
    plotfun.add_scatter(x = xpeaks , y = yval, name = fitname, line=dict(shape='spline'), mode='lines')
    print(plotfun)
    return plotfun, popt, pcov

In [1196]:
def fouriertransform(file,expt):
    "Attribute - expt - can either be 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho'"
    layout = go.Layout(xaxis=dict(title='Frequency (GHz)'),yaxis=dict(title='Amplitude'),title = 'Fourier Transform of a ' + expt.capitalize() + ' signal')
    freqPlot = go.Figure(layout=layout)
    if np.size(file) != 0:
        Data = exc.load_by_id(file)
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    xaxis = 'Time'
    if expt == 'rabi' or expt == 'ramsey':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
    elif expt == 'nmr':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms1 - y1ms0
        
    step = x1[1] - x1[0]
        
    fourTrans = np.fft.fft(y1)
    freqs = np.fft.fftfreq(y1.shape[-1], step)
    fourTransReal = fourTrans.real
    freqPlot.add_scatter(x = freqs, y = fourTransReal,line=dict(shape='spline'), mode='lines')
    print(freqPlot)
    return freqPlot

In [1219]:
experimentsList = [exp.name for exp in exc.experiments()]
lastSet = (qc.load_last_experiment()).last_data_set()
types = [{'label':'Counting', 'value': 'counting'},
         {'label':'ODMR', 'value': 'odmr'},
         {'label':'Pulsed ODMR', 'value': 'pulsedodmr'},
         {'label':'Rabi', 'value': 'rabi'},
         {'label':'Ramsey', 'value': 'ramsey'},
         {'label':'Spin Echo', 'value': 'spinecho'},
         {'label':'Double Echo', 'value': 'doubleecho'},
         {'label':'NMR', 'value': 'nmr'}]
types_analysis = [{'label':'ODMR', 'value': 'odmr'},
                  {'label':'Pulsed ODMR', 'value': 'pulsedodmr'},
                  {'label':'Rabi', 'value': 'rabi'},
                  {'label':'Ramsey', 'value': 'ramsey'},
                  {'label':'Spin Echo', 'value': 'spinecho'},
                  {'label':'Double Echo', 'value': 'doubleecho'},
                  {'label':'NMR', 'value': 'nmr'}]
dataNeeded = {'odmr': ['Frequency','Counts'], 
              'pulsedodmr': ['Frequency','Rebased_Counts'], 
              'spinecho': ['Time','Rebased_Counts'],
              'doubleecho': ['Time','Act_Counts','Ref_Counts'],
              'nmr': ['Time','Act_Counts','Ref_Counts'],
              'counting': ['Time','Rebased_Counts'],
              'rabi': ['Time','Rebased_Counts'],
              'ramsey':['Time','Rebased_Counts']}
analysisTypes = {'odmr': [{'label': 'Lorentzian', 'value': 'lorentz'}], 
                 'pulsedodmr': [{'label': 'Lorentzian', 'value': 'lorentz'}],
                 'spinecho': [{'label': 'Stretched Exponential', 'value': 'exponent'},
                              {'label': 'Fourier Transform','value': 'fourier'}],
                 'doubleecho': [{'label': 'Stretched Exponential', 'value': 'exponent'},
                              {'label': 'Fourier Transform','value': 'fourier'}],
                 'nmr': [{'label': 'Fourier Transform','value': 'fourier'}],
                 'rabi': [{'label':'Damped Sine','value':'sine'}],
                 'ramsey': [{'label': 'Fourier Transform','value': 'fourier'}]}
all_runs = [{'label': str(load_by_id(a)).split('@')[0], 'value': a} for a in range(1,lastSet.run_id)]
bounds = {'lorentz': ['Amplitude','Baseline Offset', 'ODMR Frequency (GHz)', 'Width (GHz)'],
         'sine': ['Amplitude','Baseline Offset', 'Oscillation Frequency (GHz)', 'Decay Time (ns)'],
         'exponent': ['Amplitude','Baseline Offset','Power of Stretched Exponential','Decay Time (ns)']}
default_values = {'lorentz': {'ub': [0.005,1.1,2.9,0.02], 'lb': [0.0005,1,2.6,0.007]}, 
                  'sine': {'ub': [0.1,1.2,0.008,100000], 'lb': [0.01,0.7,0.001,0]}, 
                  'exponent': {'ub': [2,3,3,5e6], 'lb': [0,0,0,0],'threshold': 0.45}}

In [1342]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    dcc.Store(id='experiment-data', storage_type='session', data = None),
    dcc.Store(id='graph-data', storage_type='session', data = None),
    dcc.Store(id='data-type-data', storage_type='session', data = None),
    dcc.Store(id='bound_data', storage_type='session', data = None),
    dcc.Store(id='analysis-graph-data', storage_type='session', data = None),
    dcc.Tabs(id="tabs", value='tab-1', children=[dcc.Tab(label='Graph', value='tab-1'),
                                                 dcc.Tab(label='Analyse', value='tab-2'),
    ]),
    html.Div(id='tabs-content')],
    style={'columnCount': 1})

@app.callback(Output('tabs-content', 'children'),
             [Input('tabs', 'value'), Input('graph-data', 'data')])
def render_content(tab, data):
    if data == None:
        if tab == 'tab-1':
            return html.Div([
                html.H2(children='Graph Data'),
                html.Div([
                    html.Label('1) Experiment'), 
                    dcc.Dropdown(id='experiment', 
                         options=[{'label': experimentsList[o], 'value': o+1} 
                                  for o in range((len(experimentsList)))]
                                ),
                    html.Label('2) Runs'), 
                    dcc.Dropdown(id='runs', multi=True),
                    html.Label('3) Data Type'),
                    dcc.Dropdown(id='data-type', options = types),
                    html.P(id='type-err', style={'color': 'red'})],
                    style={'width': '48%', 'display': 'inline-block'}), 
                html.Div([
                    html.Label('4) Extra Runs (Optional)'), 
                    dcc.Dropdown(id='extra-runs',options = all_runs, multi=True),
                    html.P(id='run-err', style={'color': 'red'}),
                    html.Div([
                        html.Label('5) Normalise? (Needed For Analysis)'), 
                        dcc.RadioItems(id='normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = 1),
                        html.Button(id='submit-button', children='Plot!')],
                        style={'width': '48%', 'display': 'inline-block'}),
                    html.Div([
                        html.Label('6) Live Plotting?'),
                        dcc.RadioItems(id='live', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = 0)
                    ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})],
                style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
            html.Hr(),
            dcc.Graph(style={'height': '700px'},id='graph', config=dict(showSendToCloud=True))
            ])
        elif tab == 'tab-2':
            return html.Div([
                html.H2(children='Analyse Data'),
                html.P(children = 'No data chosen in Graph tab!', style={'color': 'red'})
            ])
    else:
        if tab == 'tab-1':
            return html.Div([
                html.H2(children='Graph Data'),
                html.Div([
                    html.Label('1) Experiment'), 
                    dcc.Dropdown(id='experiment', 
                         options=[{'label': experimentsList[o], 'value': o+1} 
                                  for o in range((len(experimentsList)))]
                                , value = data['experiment']),
                    html.Label('2) Runs'), 
                    dcc.Dropdown(id='runs', multi=True, value = data['run']),
                    html.Label('3) Data Type'),
                    dcc.Dropdown(id='data-type', options = types, value = data['type']),
                    html.P(id='type-err', style={'color': 'red'})],
                    style={'width': '48%', 'display': 'inline-block'}), 
                html.Div([
                    html.Label('4) Extra Runs (Optional)'), 
                    dcc.Dropdown(id='extra-runs',options = all_runs, multi=True, value = data['extra-run']),
                    html.P(id='run-err', style={'color': 'red'}),
                    html.Div([
                        html.Label('5) Normalise? (Needed For Analysis)'), 
                        dcc.RadioItems(id='normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = data['normalise']),
                        html.Br(),
                        html.Button(id='submit-button', children='Plot!')],
                        style={'width': '48%', 'display': 'inline-block'}),
                    html.Div([
                        html.Label('6) Live Plotting?'),
                        dcc.RadioItems(id='live', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}])
                    ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})],
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
            html.Hr(),
            dcc.Graph(style={'height': '700px'},id='graph', config=dict(showSendToCloud=True), figure = data['graph'])
            ])
        elif tab == 'tab-2':
            if data['total-run'] == []:
                return html.Div([
                html.H2(children='Analyse Data'),
                html.P(children = 'No data chosen in Graph tab!', style={'color': 'red'})
            ])
            else:
                runs_for_analysis = ''
                for a in data['total-run']:
                    runs_for_analysis = runs_for_analysis +'['+ all_runs[a-1]['label']+'], '
                runs_for_analysis = runs_for_analysis[:-2]
                runs_options = [{'label': all_runs[a-1]['label'], 'value': a} for a in data['total-run']]
                return html.Div([
                    html.Div([
                        html.H2(children='Analyse Data'),
                        html.Div([
                            html.Label('Runs Available For Analysis:'),
                            html.P(children = runs_for_analysis),
                            html.Hr(),
                            html.Label('1) Run To Analyse'),
                            dcc.Dropdown(id='analysis-runs',options = runs_options),
                            html.Label('2) Data Type'),
                            dcc.Dropdown(id='analysis-data-type', options = types_analysis, value = data['type']),
                            html.Label('3) Analysis Type'),
                            dcc.Dropdown(id='analysis-type')],
                            style={'width': '48%', 'display': 'inline-block'}),
                        html.Div([
                            html.Label('4) Upper & Lower Bounds'),
                            html.P(id='fourier',children = None),
                            html.Div(id='bound-title',children = [
                                html.Div([
                                    html.Label('Lower:')                
                                ],style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    html.Label('Upper:')                
                                ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style = {'display': 'none'}),
                            html.Div(id = 'bound-1', children = [
                                html.Label(children = 'Amplitude'),
                                html.Div([
                                    #html.Label('Lower:'),
                                    dcc.Input(id='b1_lb', type='text', value =None)
                                ], style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    #html.Label('Upper:'),
                                    dcc.Input(id='b1_ub', type='text', value =None)
                                ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style = {'display': 'none'}),
                            html.Div(id = 'bound-2', children = [
                                html.Label(children = 'Baseline Offset'),
                                html.Div([
                                    #html.Label('Lower:'),
                                    dcc.Input(id='b2_lb', type='text', value =None)
                                ], style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    #html.Label('Upper:'),
                                    dcc.Input(id='b2_ub', type='text', value =None)
                                ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style ={'display': 'none'}),
                            html.Div(id = 'bound-3', children = [
                                html.Label(id='bound-3-label', children = 'Bound 3'),
                                html.Div([
                                    #html.Label('Lower:'),
                                    dcc.Input(id='b3_lb', type='text', value =None)
                                ], style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    #html.Label('Upper:'),
                                    dcc.Input(id='b3_ub', type='text', value =None)
                                ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style ={'display': 'none'}),
                            html.Div(id = 'bound-4', children = [
                                html.Label(id='bound-4-label', children = 'Bound 4'),
                                html.Div([
                                    #html.Label('Lower:'),
                                    dcc.Input(id='b4_lb', type='text', value =None)
                                ], style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    #html.Label('Upper:'),
                                    dcc.Input(id='b4_ub', type='text', value =None)
                                ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style ={'display': 'none'}),
                            html.Div(id='exponent-inputs',children = [
                                html.Label(children = 'Stretched Exponential Inputs'),
                                html.Div([
                                    html.Label('Threshold:'),
                                    dcc.Input(id='threshold', type='text', value =None)
                                ], style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    html.Label('Revivals?'),
                                    dcc.RadioItems(id='revivals', options = [{'label': 'Yes', 'value': 1}, 
                                                                      {'label': 'No', 'value': 0}],value=None)
                                ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                            ], style ={'display': 'none'}),
                            html.Br(),
                            html.Br(),
                            html.Div([
                                html.Div([
                                    html.Button(id='reset_button', children='Reset!')],      
                                    style={'width': '48%', 'display': 'inline-block'}),
                                html.Div([
                                    html.Button(id='analyse_button', children='Analyse!'),
                                    html.P(id ='a-error',children=None, style={'color': 'red'})],
                                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'})     
                            ],style={'width': '100%','display': 'inline-block'}),
                            ], 
                            style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
                    ],style = {'width': '100%', 'display': 'inline-block'}),
                    html.Hr(),
                    dcc.Graph(style={'height': '700px'},id='analysis-graph', figure = data['graph'],
                              config=dict(showSendToCloud=True)),
                    html.Hr(),
                    dcc.Graph(style={'height': '700px','width': '1270px','display': 'none'},id='fourier-graph', 
                              figure = [],config=dict(showSendToCloud=True))
                ])

@app.callback(
    [Output('runs', 'options'),Output('experiment-data','data')],
    [Input('experiment', 'value')],
    [State('experiment-data','data')])
def update_run(selected_experiment, data):
    if selected_experiment is None:
        raise PreventUpdate
    exper = load_experiment(selected_experiment)
    data = data or {'experiment': None, 'run': None, 'type': None, 'extra-run': None, 'normalise': 1, 'graph': None, 'total-run': None}
    data['experiment'] = selected_experiment
    return [{'label': str(exper.data_set(a+1)).split('@')[0], 
             'value': exper.data_set(a+1).run_id} for a in range(exper.last_counter)], data

@app.callback(
    [Output('graph', 'figure'), Output('type-err','children'), 
     Output('run-err','children'), Output('graph-data','data')],
    [Input('submit-button','n_clicks')],
    [State('runs','value'), State('data-type','value'), 
     State('extra-runs','value'), State('normalise', 'value'), 
     State('experiment-data','data')])
def update_graph(clicks, selected_runs, selected_type, extra_runs, normalised, data):
    if clicks is None:
        raise PreventUpdate
    if selected_runs is None:
        raise PreventUpdate
    for a in selected_runs:
        params = str(load_by_id(a).parameters).split(',')
        for b in dataNeeded[selected_type]:
            if (b in params) is False:
                return dash.no_update, 'Wrong Data Type!', dash.no_update, dash.no_update
    total_runs = selected_runs
    if extra_runs != None:
        for c in extra_runs:
            params_2 = str(load_by_id(c).parameters).split(',')
            for d in dataNeeded[selected_type]:
                if (d in params_2) is False:
                    return dash.no_update, dash.no_update, 'Wrong Run!', dash.no_update
        total_runs = total_runs + extra_runs
    totaldata = plotdata(selected_type, total_runs, normalised)
    data['run'] = selected_runs
    data['type'] = selected_type
    data['extra-run'] = extra_runs
    data['normalise'] = normalised
    data['total-run'] = total_runs
    data['graph'] = totaldata
    return totaldata, None, None, data

@app.callback(
    [Output('analysis-type', 'options'), Output('analysis-type','value'), 
     Output('data-type-data','data')],
    [Input('analysis-data-type', 'value')],
    [State('data-type-data','data')])
def update_datatype(selected_data_type, data):
    if selected_data_type is None:
        raise PreventUpdate
    analysis = analysisTypes[selected_data_type]
    analysisValue = analysis[0]['value']
    data = data or {'run': None, 'data-type': None, 'analysis-type': None, 'graph': None}
    data['data-type'] = selected_data_type
    data['analysis-type'] = analysisValue
    return analysis, analysisValue, data


@app.callback(
    [Output('bound-title','style'),Output('bound-1','style'),Output('bound-2','style'),
     Output('bound-3','style'),Output('bound-4','style'), 
     Output('exponent-inputs','style'), Output('fourier','children'),
     Output('bound-3-label','children'),Output('bound-4-label','children'),
     Output('b1_ub','value'),Output('b1_lb','value'),Output('b2_ub','value'),
     Output('b2_lb','value'),Output('b3_ub','value'),Output('b3_lb','value'),
     Output('b4_ub','value'),Output('b4_lb','value'),Output('threshold','value'),
     Output('revivals','value'),Output('fourier-graph','style')],
    [Input('analysis-type','value'),Input('reset_button','n_clicks')])
def update_bounds(a_type,clicks):
    if a_type is None:
        raise PreventUpdate
    if a_type == 'fourier':
        return ({'display': 'none'},{'display': 'none'},{'display': 'none'}, {'display': 'none'},{'display': 'none'},
                {'display': 'none'},'No Bounds Necessary.', '','',None,None,None,None,None,None,None,None,None,None, 
                {'height': '700px','display': 'inline-block'})
    if a_type == 'exponent':
        return ({'display': 'inline'},{'display': 'inline'}, {'display': 'inline'}, {'display': 'inline'},
                {'display': 'inline'},{'display': 'inline'}, '','Power of Stretched Exponential', 
                'Decay Time (ns)',default_values[a_type]['ub'][0],default_values[a_type]['lb'][0],
                default_values[a_type]['ub'][1],default_values[a_type]['lb'][1],default_values[a_type]['ub'][2],
                default_values[a_type]['lb'][2],default_values[a_type]['ub'][3],default_values[a_type]['lb'][3],
                default_values[a_type]['threshold'],0,{'height': '700px','display': 'none'})
    if a_type == 'lorentz':
        return ({'display': 'inline'},{'display': 'inline'}, {'display': 'inline'}, {'display': 'inline'}, 
                {'display': 'inline'}, {'display': 'none'}, '', 'ODMR Frequency (GHz)','Width (GHz)',
                default_values[a_type]['ub'][0],default_values[a_type]['lb'][0],default_values[a_type]['ub'][1],
                default_values[a_type]['lb'][1],default_values[a_type]['ub'][2],default_values[a_type]['lb'][2],
                default_values[a_type]['ub'][3],default_values[a_type]['lb'][3],0,None,{'height': '700px','display': 'none'})
    if a_type == 'sine':
        return ({'display': 'inline'},{'display': 'inline'},{'display': 'inline'},{'display': 'inline'},
                {'display': 'inline'},{'display': 'none'}, '', 'Oscillation Frequency (GHz)', 'Decay Time (ns)',
                default_values[a_type]['ub'][0],default_values[a_type]['lb'][0],default_values[a_type]['ub'][1],
                default_values[a_type]['lb'][1],default_values[a_type]['ub'][2],default_values[a_type]['lb'][2],
                default_values[a_type]['ub'][3],default_values[a_type]['lb'][3],0,None,{'height': '700px','display': 'none'})    
    
@app.callback(
    Output('bound_data','data'),
    [Input('threshold','value'), Input('b1_ub','value'), Input('b1_lb','value'),
     Input('b2_ub','value'), Input('b2_lb','value'), Input('b3_ub','value'),
     Input('b3_lb','value'), Input('b4_ub','value'), Input('b4_lb','value'),
     Input('revivals','value')],  
    [State('bound_data','data')])
def store_bounds(threshold,b1_ub,b1_lb,b2_ub,b2_lb,b3_ub,b3_lb,b4_ub,b4_lb,revival,data):
    data = data or {'ub': None, 'lb': None, 'threshold': None, 'revival': 0}
    ub = [b1_ub,b2_ub,b3_ub,b4_ub]
    lb = [b1_lb,b2_lb,b3_lb,b4_lb]
    data['ub'] = ub
    data['lb'] = lb
    data['threshold'] = threshold
    data['revival'] = revival
    return data
    
@app.callback(
    [Output('analysis-graph', 'figure'), Output('analysis-graph-data','data'),
     Output('fourier-graph','figure'), Output('a-error','children')],
    [Input('analyse_button','n_clicks')],
    [State('analysis-runs','value'), State('analysis-data-type', 'value'),
     State('analysis-type','value'), State('analysis-graph','figure'),
     State('bound_data','data')])
def update_analysis_graph(clicks, selected_run, data_type, analysis_type, graph, data):

    if clicks is None:
        raise PreventUpdate
    if selected_run is None:
        return dash.no_update, dash.no_update, dash.no_update, 'No Run Chosen!'
    if analysis_type == 'fourier':
        plot_fourier = fouriertransform(selected_run,data_type)
        return graph, data, plot_fourier, None
    elif analysis_type == 'lorentz':
        totaldata = fitlorentzian(selected_run, data_type, go.Figure(graph),data['lb'],data['ub'])
        return totaldata[0], data, [], None
    elif analysis_type == 'exponent':
        totaldata = find_T2(selected_run, data_type, go.Figure(graph),data['threshold'],data['lb'],data['ub'],data['revival'])
        return totaldata[0], data, [], None
    elif analysis_type == 'sine':
        totaldata = fitsinedamp(selected_run, data_type, go.Figure(graph),data['lb'],data['ub'])
        return totaldata[0], data, [], None

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Sep/2019 14:20:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566376285 HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1567160754 HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:20:04] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566376287 HTTP/1.1

[0.0005, 1, 2.6, 0.007] [0.005, 1.1, 2.9, 0.02]
Figure({
    'data': [{'mode': 'lines+markers',
              'name': 'cwODMR_power20_B_111_48G_correctedField_NV_1_ #1',
              'type': 'scatter',
              'x': [2700000000, 2701000000, 2702000000, 2703000000, 2704000000,
                    2705000000, 2706000000, 2707000000, 2708000000, 2709000000,
                    2710000000, 2711000000, 2712000000, 2713000000, 2714000000,
                    2715000000, 2716000000, 2717000000, 2718000000, 2719000000,
                    2720000000, 2721000000, 2722000000, 2723000000, 2724000000,
                    2725000000, 2726000000, 2727000000, 2728000000, 2729000000,
                    2730000000, 2731000000, 2732000000, 2733000000, 2734000000,
                    2735000000, 2736000000, 2737000000, 2738000000, 2739000000,
                    2740000000, 2741000000, 2742000000, 2743000000, 2744000000,
                    2745000000, 2746000000, 2747000000, 2748000000, 274900000

127.0.0.1 - - [05/Sep/2019 14:35:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:35:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.

[0.0005, 1, 2.6, 0.007] [0.005, 1.1, 2.9, 0.02]
Figure({
    'data': [{'mode': 'lines+markers',
              'name': 'cwODMR_power20_B_111_48G_correctedField_NV_1_ #1',
              'type': 'scatter',
              'visible': 'legendonly',
              'x': [2700000000, 2701000000, 2702000000, 2703000000, 2704000000,
                    2705000000, 2706000000, 2707000000, 2708000000, 2709000000,
                    2710000000, 2711000000, 2712000000, 2713000000, 2714000000,
                    2715000000, 2716000000, 2717000000, 2718000000, 2719000000,
                    2720000000, 2721000000, 2722000000, 2723000000, 2724000000,
                    2725000000, 2726000000, 2727000000, 2728000000, 2729000000,
                    2730000000, 2731000000, 2732000000, 2733000000, 2734000000,
                    2735000000, 2736000000, 2737000000, 2738000000, 2739000000,
                    2740000000, 2741000000, 2742000000, 2743000000, 2744000000,
                    2745000000, 274600

127.0.0.1 - - [05/Sep/2019 14:36:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:36:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:36:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:36:36] "POST /_dash-update-component HTTP/1.1" 200 -


[0.0005, 1, 2.6, 0.007] [0.005, 1.1, 2.9, '0.008']
Figure({
    'data': [{'mode': 'lines+markers',
              'name': 'cwODMR_power20_B_111_48G_correctedField_NV_1_ #1',
              'type': 'scatter',
              'visible': 'legendonly',
              'x': [2700000000, 2701000000, 2702000000, 2703000000, 2704000000,
                    2705000000, 2706000000, 2707000000, 2708000000, 2709000000,
                    2710000000, 2711000000, 2712000000, 2713000000, 2714000000,
                    2715000000, 2716000000, 2717000000, 2718000000, 2719000000,
                    2720000000, 2721000000, 2722000000, 2723000000, 2724000000,
                    2725000000, 2726000000, 2727000000, 2728000000, 2729000000,
                    2730000000, 2731000000, 2732000000, 2733000000, 2734000000,
                    2735000000, 2736000000, 2737000000, 2738000000, 2739000000,
                    2740000000, 2741000000, 2742000000, 2743000000, 2744000000,
                    2745000000, 274

127.0.0.1 - - [05/Sep/2019 14:36:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:36:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 14:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.